In [1]:
import pandas as pd
import numpy as np
import os 
import pickle
#from fuzzywuzzy import process, fuzz
from rapidfuzz import process, utils, fuzz
from collections import Counter
from tqdm import tqdm
pd.options.display.max_columns = None

# checking the dataset's data
Checking the data from the [given](https://www.kaggle.com/datasets/andrewmvd/spotify-playlists?select=spotify_dataset.csv) database.

In [2]:
data = pd.read_csv('../data/charts.csv', sep = ',', on_bad_lines='skip')
print(data.shape)
data.head()

(26173514, 9)


,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [3]:
data[['region', 'chart']].value_counts()

region         chart  
Brazil         top200     364516
Argentina      top200     364385
United States  top200     364184
Austria        top200     363925
Australia      top200     362189
                           ...  
India          viral50     51688
Ukraine        viral50     26778
Russia         viral50     26069
South Korea    viral50     16700
Luxembourg     top200       8962
Length: 139, dtype: int64

For my analysis, I will consider one of the regions given by the dataset. In this case, Spain.

In [4]:
data[data['region'] == 'Spain'][['region', 'chart']].value_counts()

region  chart  
Spain   top200     358355
        viral50     90904
dtype: int64

In [5]:
data_spain = data[(data['region'] == 'Spain')]

check if data has duplicated values

In [6]:
data_spain.columns

Index(['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend',
       'streams'],
      dtype='object')

In [7]:
data_spain.duplicated().sum()

0

In [8]:
nulos = (data_spain.isnull().sum() / data_spain.shape[0] * 100).reset_index()
nulos = nulos[nulos[0] > 0]
nulos.columns = ["columnas", "%_nulos"]
nulos

,columnas,%_nulos
8,streams,20.234208


No need to remove columns, all have values.

In [9]:
data_spain.head()

,title,rank,date,artist,url,region,chart,trend,streams
6844,Alone,65,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Spain,top200,MOVE_UP,25347.0
7045,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Spain,top200,SAME_POSITION,189721.0
7046,Reggaetón Lento (Bailemos),2,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Spain,top200,SAME_POSITION,165291.0
7047,Safari,3,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Spain,top200,SAME_POSITION,141575.0
7048,Vente Pa' Ca (feat. Maluma),4,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Spain,top200,SAME_POSITION,119865.0


# Export
I will export the whole dataset and generate a dataset with the tracks and artist without duplicated rows.

In [40]:
with open('../data/pickle/dataset_spain_base.pickle', 'wb') as data:
    pickle.dump(data_spain, data)

# Generate clean artist - track data
I will now get a unique list from the track - artist relation, in order to make api calls.

In [54]:
data_spain.head()

,title,rank,date,artist,url,region,chart,trend,streams,artist_clean,track_clean,artist_clean_new,check,track_clean_new
6844,Alone,65,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Spain,top200,MOVE_UP,25347.0,alan walker,alone,alan walker,same,alone
7045,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Spain,top200,SAME_POSITION,189721.0,shakira,chantaje (feat. maluma),shakira,same,chantaje (feat. maluma)
7046,Reggaetón Lento (Bailemos),2,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Spain,top200,SAME_POSITION,165291.0,cnco,reggaetón lento (bailemos),cnco,same,reggaetón lento (bailemos)
7047,Safari,3,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Spain,top200,SAME_POSITION,141575.0,"j balvin, pharrell williams, bia, sky",safari,"j balvin, pharrell williams, bia, sky",same,safari
7048,Vente Pa' Ca (feat. Maluma),4,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Spain,top200,SAME_POSITION,119865.0,ricky martin,vente pa' ca (feat. maluma),ricky martin,same,vente pa' ca (feat. maluma)


In [56]:
data_spain = data_spain[['title', 'artist', 'url']]
print(data_spain.shape)
data_spain.head()

(449259, 3)


,title,artist,url
6844,Alone,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...
7045,Chantaje (feat. Maluma),Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...
7046,Reggaetón Lento (Bailemos),CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...
7047,Safari,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...
7048,Vente Pa' Ca (feat. Maluma),Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...


In [57]:
data_spain.drop_duplicates(inplace=True)
print(data_spain.shape)
data_spain.head()

(10742, 3)


,title,artist,url
6844,Alone,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...
7045,Chantaje (feat. Maluma),Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...
7046,Reggaetón Lento (Bailemos),CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...
7047,Safari,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...
7048,Vente Pa' Ca (feat. Maluma),Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...


In [58]:
print('unique artists:',len(data_spain['artist'].unique().tolist()),'; unique songs (with unique names):',len(data_spain['title'].unique().tolist()))

unique artists: 5801 ; unique songs (with unique names): 9009


making all `artist` lowercase, removing all spaces and weird characters

In [59]:
data_spain['artist_clean'] = data_spain['artist'].str.strip().str.lower()
print(len(data_spain['artist_clean'].unique().tolist()), len(data_spain['artist'].unique().tolist()))

5797 5801


doing the same with the `tracks`

In [60]:
data_spain['track_clean'] = data_spain['title'].str.strip().str.lower()
print(len(data_spain['track_clean'].unique().tolist()), len(data_spain['title'].unique().tolist()))

8863 9009


Fixing the `artist`s, in order to remove duplicated artist or typos -- replacing by the most common occurance by similarity.

In [61]:
def parecido_artistas(col, dict_artistas):   
    maximo = 0
    for key in dict_artistas.keys():
    #print(key)
        #print(genero)
        comparo = fuzz.ratio(key, col)
        if comparo > maximo:
            maximo = comparo
            resultado = key
    if maximo > 90:
        return resultado
    else:
        return col

In [62]:
data_spain[data_spain['artist_clean'].str.contains('shakira') == True]['artist_clean'].unique().tolist()[:10]

['shakira',
 'carlos vives, shakira',
 'pablo alborán, david bisbal, antonio carmona, manuel carrasco, jesse & joy, juanes, pablo lópez, malú, vanesa martín, india martinez, antonio orozco, niña pastori, laura pausini, abel pintos, rozalén, shakira, tommy torres',
 'prince royce, shakira',
 'shakira, anuel aa',
 'shakira, maluma',
 'pablo alborán, david bisbal, antonio carmona, jesse & joy, juanes, pablo lópez, malú, vanesa martín, india martinez, antonio orozco, niña pastori, laura pausini, abel pintos, rozalén, shakira, tommy torres, manuel carrasco',
 'camilo, shakira, pedro capó',
 'black eyed peas, shakira']

In [63]:
fuzz.ratio(
    'pablo alborán, david bisbal, antonio carmona, manuel carrasco, jesse & joy, juanes, pablo lópez, malú, vanesa martín, india martinez, antonio orozco, niña pastori, laura pausini, abel pintos, rozalén, shakira, tommy torres', 
    'pablo alborán, david bisbal, antonio carmona, jesse & joy, juanes, pablo lópez, malú, vanesa martín, india martinez, antonio orozco, niña pastori, laura pausini, abel pintos, rozalén, shakira, tommy torres, manuel carrasco')

92.34234234234235

In [64]:
fuzz.ratio(
    'shakira, anuel aa',
 'shakira, maluma',)

75.0

In [65]:
cuenta_artistas = Counter(artistas for artistas in data_spain['artist_clean'])
print(len(cuenta_artistas))
cuenta_artistas

5797


Counter({'alan walker': 2,
         'shakira': 9,
         'cnco': 8,
         'j balvin, pharrell williams, bia, sky': 2,
         'ricky martin': 4,
         'carlos vives, shakira': 5,
         'clean bandit': 7,
         'sebastian yatra': 15,
         'charly black, daddy yankee': 1,
         'zion & lennox': 5,
         'bruno mars': 4,
         'dj snake, justin bieber': 1,
         'jacob forever': 4,
         'iamchino, el chacal': 1,
         'the weeknd, daft punk': 3,
         'wisin': 5,
         'enrique iglesias': 8,
         'maluma': 30,
         'the chainsmokers, halsey': 1,
         'cosculluela': 5,
         'morat': 21,
         'maroon 5, kendrick lamar': 1,
         'juan magán': 4,
         'alejandro sanz, marc anthony': 1,
         'xantos, dynell': 3,
         'juanes': 10,
         'martin garrix, bebe rexha': 1,
         'drake, wizkid, kyla': 1,
         'ozuna': 42,
         'don omar, zion & lennox': 1,
         'chino & nacho, daddy yankee': 1,
       

In [66]:
dict_artistas = dict(cuenta_artistas)
dict_artistas = {k:v for k,v in dict_artistas.items() if v > 1}
print(len(dict_artistas))
dict_artistas#.get(3)

1363


{'alan walker': 2,
 'shakira': 9,
 'cnco': 8,
 'j balvin, pharrell williams, bia, sky': 2,
 'ricky martin': 4,
 'carlos vives, shakira': 5,
 'clean bandit': 7,
 'sebastian yatra': 15,
 'zion & lennox': 5,
 'bruno mars': 4,
 'jacob forever': 4,
 'the weeknd, daft punk': 3,
 'wisin': 5,
 'enrique iglesias': 8,
 'maluma': 30,
 'cosculluela': 5,
 'morat': 21,
 'juan magán': 4,
 'xantos, dynell': 3,
 'juanes': 10,
 'ozuna': 42,
 'daddy yankee': 11,
 'major lazer': 10,
 'justin timberlake': 7,
 'david bisbal': 4,
 'melendi': 29,
 'calvin harris': 9,
 'reik': 5,
 'twenty one pilots': 8,
 'j balvin': 31,
 'sia': 12,
 'the chainsmokers': 13,
 'jonas blue, jp cooper': 3,
 'charlie puth': 8,
 'james arthur': 4,
 'shawn mendes': 26,
 'piso 21': 9,
 'gente de zona': 4,
 'mariah carey': 4,
 'alvaro soler': 14,
 'jennifer lopez': 4,
 'nicky jam': 6,
 'kevin roldan': 4,
 'joey montana': 2,
 'carlos baute': 5,
 'anuel aa': 36,
 'justin quiles': 13,
 'justin bieber': 23,
 'reykon': 6,
 'raphael': 7,
 'd

In [67]:
data_test = data_spain.sample(1000)

In [68]:
data_test['artist'] = data_test.apply(lambda x: parecido_artistas(x['artist_clean'], dict_artistas), axis=1)
data_test[['artist_clean','artist']].value_counts()

artist_clean          artist              
ariana grande         ariana grande           8
adele                 adele                   6
beret                 beret                   6
maluma                maluma                  5
lady gaga             lady gaga               5
                                             ..
foo fighters          foo fighters            1
frathos, haze         frathos, haze           1
fresquito, mango      fresquito, mango        1
friedrich habetler    friedrich habetler      1
ñengo flow, anuel aa  ñengo flow, anuel aa    1
Length: 840, dtype: int64

In [69]:
tqdm.pandas()

data_spain['artist_clean_new'] = data_spain.progress_apply(lambda x: parecido_artistas(x['artist_clean'], dict_artistas), axis=1)
data_spain[['artist_clean','artist_clean_new']].value_counts()

100%|██████████| 10742/10742 [00:06<00:00, 1559.04it/s]


artist_clean                                       artist_clean_new                                 
bad bunny                                          bad bunny                                            57
ariana grande                                      ariana grande                                        54
taylor swift                                       taylor swift                                         53
ed sheeran                                         ed sheeran                                           52
c. tangana                                         c. tangana                                           47
                                                                                                        ..
gorillaz, shelley fka dram                         gorillaz, shelley fka dram                            1
goorgo, shooter                                    goorgo, shooter                                       1
googie rene                                

In [70]:
data_spain['check'] = np.where(data_spain['artist_clean_new'] == data_spain['artist_clean'], 'same', 'different' )
data_spain['check'].value_counts()

same         10729
different       13
Name: check, dtype: int64

In [71]:
data_spain[data_spain['check']== 'different']

,title,artist,url,artist_clean,track_clean,artist_clean_new,check
146442,"Mi Gente (Electronic Remix) - Aleteo, Zapateo ...",J Balvinn,https://open.spotify.com/track/2A3cP26AnUDdn7q...,j balvinn,"mi gente (electronic remix) - aleteo, zapateo ...",j balvin,different
4284613,Rebota,Guaynaa,https://open.spotify.com/track/2YRvdf9cdS88y1w...,guaynaa,rebota,guayna,different
7011635,Quizas,"Rich Music LTD, Sech, Dalex, Justin Quiles, Le...",https://open.spotify.com/track/5Id5B3dxJZhPcV9...,"rich music ltd, sech, dalex, justin quiles, le...",quizas,"rich music ltd, sech, dalex, justin quiles, le...",different
11823872,Don't Dream It's Over - Operación Triunfo 2017,"MARINA, Alfred García",https://open.spotify.com/track/7A6Xg6DoqYkD585...,"marina, alfred garcía",don't dream it's over - operación triunfo 2017,"amaia, alfred garcía",different
12245747,Eres Mia,Bad Buny,https://open.spotify.com/track/7zCpiKnNs8ycXt8...,bad buny,eres mia,bad bunny,different
12349241,Let Me Love You - Operación Triunfo 2017,"Raoul Vázquez, Aitana",https://open.spotify.com/track/6pWvGqejH5Cjq3l...,"raoul vázquez, aitana",let me love you - operación triunfo 2017,"raoul vázquez, marina",different
12802395,Hace Cuánto,"Juancho Marqués, Lÿ, Gabriel Fernández",https://open.spotify.com/track/28tfmO9J7HbWkaR...,"juancho marqués, lÿ, gabriel fernández",hace cuánto,"juancho marqués, gabriel fernández",different
13405548,Me Hace Tanta Falta,Bryann Myers,https://open.spotify.com/track/2Wuy73xtJpRHyZy...,bryann myers,me hace tanta falta,bryant myers,different
14135335,Tu Te Imaginas (Tti),Raw Alejandro,https://open.spotify.com/track/6mDj7JNb9XpFdIF...,raw alejandro,tu te imaginas (tti),rauw alejandro,different
20583902,Mi Pan Su Su Sum,DJ Krass,https://open.spotify.com/track/2CUrX5Vrc3OLFQo...,dj krass,mi pan su su sum,dj kass,different


Doing the same process with the `tracks`, in order to remove typos.

In [72]:
data_spain.head()

,title,artist,url,artist_clean,track_clean,artist_clean_new,check
6844,Alone,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,alan walker,alone,alan walker,same
7045,Chantaje (feat. Maluma),Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,shakira,chantaje (feat. maluma),shakira,same
7046,Reggaetón Lento (Bailemos),CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,cnco,reggaetón lento (bailemos),cnco,same
7047,Safari,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,"j balvin, pharrell williams, bia, sky",safari,"j balvin, pharrell williams, bia, sky",same
7048,Vente Pa' Ca (feat. Maluma),Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,ricky martin,vente pa' ca (feat. maluma),ricky martin,same


In [73]:
data_spain[data_spain['track_clean'].str.contains("atrévete") == True]['track_clean'].unique().tolist()[:10]

['atrévete-te-te', 'atrévete']

In [74]:
fuzz.ratio(
    'atrévete-te-te', 'atrévete') ## the fuzz ratio needs to be bigger than this value

72.72727272727273

In [75]:
cuenta_tracks = Counter(track for track in data_spain['track_clean'])
print(len(cuenta_tracks))
cuenta_tracks

8863


Counter({'alone': 2,
         'chantaje (feat. maluma)': 1,
         'reggaetón lento (bailemos)': 1,
         'safari': 2,
         "vente pa' ca (feat. maluma)": 1,
         'la bicicleta': 4,
         'rockabye (feat. sean paul & anne-marie)': 1,
         'traicionera': 3,
         'gyal you a party animal - remix': 1,
         'otra vez (feat. j balvin)': 1,
         '24k magic': 1,
         'let me love you': 1,
         'hasta que se seque el malecón (feat. farruko) - remix': 1,
         'ay mi dios (feat. pitbull, yandel & chacal)': 1,
         'starboy': 2,
         'vacaciones': 2,
         'duele el corazon (feat. wisin)': 1,
         'borro cassette': 1,
         'closer': 1,
         'manicomio': 1,
         'cómo te atreves': 2,
         "don't wanna know": 1,
         'quiero que sepas': 1,
         'deja que te bese': 1,
         'bailame despacio': 1,
         'fuego': 10,
         'in the name of love': 1,
         'one dance': 1,
         'dile que tu me quieres': 2,


In [76]:
dict_tracks = dict(cuenta_tracks)
dict_tracks = {k:v for k,v in dict_tracks.items() if v > 4}
print(len(dict_tracks))
dict_tracks

47


{'fuego': 10,
 'sorry': 5,
 'all night': 5,
 'bella': 5,
 'me rehúso': 6,
 'vuelve': 8,
 'shape of you': 6,
 'what lovers do (feat. sza)': 6,
 'loca': 5,
 'heaven': 5,
 'issues': 5,
 'thunder': 8,
 'y, ¿si fuera ella? - + es +': 6,
 'bonita': 8,
 'believer': 6,
 'mama': 5,
 'despacito': 6,
 'parís': 6,
 'scooby doo pa pa': 5,
 'contigo': 9,
 'deja vu': 6,
 'conmigo': 5,
 'crazy': 5,
 'barcelona': 5,
 'loco': 7,
 'me gusta': 5,
 'vete': 5,
 'bésame': 5,
 'lo siento': 5,
 'si te vas': 5,
 'ella': 7,
 'lo malo': 5,
 'la cintura': 6,
 'veneno': 6,
 'stay': 6,
 'asesina': 5,
 'lucid dreams': 5,
 'lento': 8,
 'better': 6,
 '911': 5,
 'home': 5,
 'circles': 5,
 'feliz navidad': 5,
 'noche de paz': 5,
 'campana sobre campana': 5,
 'goodbye': 5,
 'this is halloween': 5}

In [81]:
def parecido_tracks(col, dict_tracks):   
    maximo = 0
    for key in dict_tracks.keys():
    #print(key)
        #print(genero)
        comparo = fuzz.ratio(key, col)
        if comparo > maximo:
            maximo = comparo
            resultado = key
    if maximo > 90:
        return resultado
    else:
        return col

In [82]:
tqdm.pandas()

data_spain['track_clean_new'] = data_spain.progress_apply(lambda x: parecido_tracks(x['track_clean'], dict_tracks), axis=1)
data_spain[['track_clean','track_clean_new']].value_counts()

100%|██████████| 10742/10742 [00:00<00:00, 35486.02it/s]


track_clean                   track_clean_new             
fuego                         fuego                           10
contigo                       contigo                          9
thunder                       thunder                          8
lento                         lento                            8
vuelve                        vuelve                           8
                                                              ..
hand it over                  hand it over                     1
halo                          halo                             1
hallucinate                   hallucinate                      1
halloween theme - main title  halloween theme - main title     1
항상 (hangsang)                 항상 (hangsang)                    1
Length: 8863, dtype: int64

In [83]:
data_spain['check'] = np.where(data_spain['track_clean_new'] == data_spain['track_clean'], 'same', 'different')
data_spain['check'].value_counts()

same         10734
different        8
Name: check, dtype: int64

In [84]:
data_spain[data_spain['check'] == 'different']

,title,artist,url,artist_clean,track_clean,artist_clean_new,check,track_clean_new
23340,Scooby Doo Papa!,Lean Trap,https://open.spotify.com/track/6csZYoffpZ7iuSw...,lean trap,scooby doo papa!,lean trap,different,scooby doo pa pa
8636200,Despacio,"Natti Natasha, Nicky Jam, Manuel Turizo, Myke ...",https://open.spotify.com/track/203uubEQA07to1k...,"natti natasha, nicky jam, manuel turizo, myke ...",despacio,"natti natasha, nicky jam, manuel turizo, myke ...",different,despacito
9221291,Me Gustas,Kevin Roldan,https://open.spotify.com/track/2d8rQXA25Yk5ZE3...,kevin roldan,me gustas,kevin roldan,different,me gusta
11991420,Goodbyes,Mafalda,https://open.spotify.com/track/28VN0jYZc6Iyror...,mafalda,goodbyes,mafalda,different,goodbye
12288911,Scooby Do Papa,LA TROPA ESTRELLA,https://open.spotify.com/track/7z40c1h4BcXMSEL...,la tropa estrella,scooby do papa,la tropa estrella,different,scooby doo pa pa
13353113,Me Gustas,Tutto Duran,https://open.spotify.com/track/59LWNzlcxkc2kym...,tutto duran,me gustas,tutto duran,different,me gusta
13891679,Bellas,"Anthony Santos, Romeo Santos",https://open.spotify.com/track/5MSlG8Mg7OQkgJ0...,"anthony santos, romeo santos",bellas,"anthony santos, romeo santos",different,bella
24009813,Believe,Eminem,https://open.spotify.com/track/2XDhuJvvx2wcIRO...,eminem,believe,eminem,different,believer


In [86]:
print(len(data_spain['track_clean_new'].unique().tolist()), len(data_spain['title'].unique().tolist()))

8856 9009


# Conversion
I will save the input file as .pkl as the original one is too big for github.

In [88]:
data_final = data_spain.drop_duplicates(subset=['artist_clean_new', 'track_clean_new', 'url'])
print(data_final.shape)
data_final.head()

(10740, 8)


,title,artist,url,artist_clean,track_clean,artist_clean_new,check,track_clean_new
6844,Alone,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,alan walker,alone,alan walker,same,alone
7045,Chantaje (feat. Maluma),Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,shakira,chantaje (feat. maluma),shakira,same,chantaje (feat. maluma)
7046,Reggaetón Lento (Bailemos),CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,cnco,reggaetón lento (bailemos),cnco,same,reggaetón lento (bailemos)
7047,Safari,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,"j balvin, pharrell williams, bia, sky",safari,"j balvin, pharrell williams, bia, sky",same,safari
7048,Vente Pa' Ca (feat. Maluma),Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,ricky martin,vente pa' ca (feat. maluma),ricky martin,same,vente pa' ca (feat. maluma)


In [89]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'wb') as data_nowplaying:
    pickle.dump(data_final, data_nowplaying)